# Deep Fool

Experiments with the DeepFool algorithm from [Moosavi-Dezfooli, Frossard](https://github.com/LTS4/DeepFool/tree/master/Python). In order to use this notebook the DeepFool implementation has to be downloaded and the Python folder saved to `modules`.

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torchvision import models
from torchvision import transforms

from IPython.display import clear_output

# Import helper functions
from modules.helper import *
from modules.dataset import ImageNetSubset

## 0. Load Model and Data

We load the pre-trained googleNet and the data. We then remove the softmax output layer as required.

In [23]:
# Get googleNet without the softmax layer

# Load pretrained net
model = models.googlenet(pretrained=True)

# Remove output layer
modules = list(model.children())[:-1]
googleNet = nn.Sequential(*modules)

for p in googleNet.parameters():
    p.requires_grad = False
    
googleNet.eval();

In [25]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]   

preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

data_loader = torch.utils.data.DataLoader(
    ImageNetSubset("data/ImageNet_subset//dev_dataset.csv", "data/ImageNet_subset//images/", transform=preprocess))

In [30]:
sample = 766

image_clean, class_index = data_loader.dataset[sample]
image_clean.unsqueeze_(0)
class_index.unsqueeze_(0)



pred_classes, conf, _ = predict(model, image_clean, class_index)

In [32]:
## The output of the netwok minus softmax is of dimension (1024, 1, 1).

sample = 766

image_clean, class_index = data_loader.dataset[sample]
image_clean.unsqueeze_(0)
class_index.unsqueeze_(0)


with torch.no_grad():
    prediction = googleNet(image_clean)  

# Get class index and confidence for prediction 
prediction = torch.nn.functional.softmax(prediction[0].cpu().detach(), dim=0).numpy()

In [34]:
prediction.shape

(1024, 1, 1)